# Exercise 0 - Camera Calibration
The original notebook is [here](https://colab.research.google.com/github/YoniChechik/AI_is_Math/blob/master/c_07_camera_calibration/multi_plane_calib.ipynb)



In [ ]:
# JUST TUN THIS CELL

from glob import glob

import cv2
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# JUST TUN THIS CELL

img_mask = "./images/*.jpeg"

# Checkerboard size and pattern (number of inner corners per a chessboard row and column)
# These values are critical for calibration, since they define the 3D points in the real world.
checkerboard_square_size = 2.88
checkerboard_pattern = (9, 6)


In [ ]:
# JUST TUN THIS CELL

img_names = glob(img_mask)
num_images = len(img_names)

world_checkerboard_corners = np.zeros((np.prod(checkerboard_pattern), 3), np.float32)
world_checkerboard_corners[:, :2] = np.indices(checkerboard_pattern).T.reshape(-1, 2)
world_checkerboard_corners *= checkerboard_square_size

world_points = []
image_pixels = []


## Step 1: find all corners in calibration plane


In [ ]:
# JUST TUN THIS CELL

plt.figure(figsize=(20, 20))

for i, fn in enumerate(img_names):
    imgBGR = cv2.imread(fn, cv2.IMREAD_COLOR)

    if imgBGR is None:
        print("Failed to load", fn)
        continue

    h, w = imgBGR.shape[:2]

    imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(imgRGB, cv2.COLOR_RGB2GRAY)

	# find the chessboard corners in the image
    found, image_checkerboard_corners = cv2.findChessboardCorners(img, checkerboard_pattern)

    if not found:
        print("chessboard not found")
        continue

    if i < 12:
        # visualize the corners
        out_image = cv2.drawChessboardCorners(imgRGB, checkerboard_pattern, image_checkerboard_corners, found)
        plt.subplot(4, 3, i + 1)
        plt.imshow(out_image)

    image_pixels.append(image_checkerboard_corners.reshape(-1, 2))
    world_points.append(world_checkerboard_corners)


plt.show()


## Step 2: estimate camera intrinsics + distortion coefficients
You will also get extrinsic rotation and translation vectors per image. Rotation vector is another representation for a full R matrix
(more on it here: https://en.wikipedia.org/wiki/Rodrigues%27_rotation_formula)


In [ ]:
# JUST TUN THIS CELL

# calculate camera parameters
error, camera_matrix, dist_coefs, _rvecs, _tvecs = cv2.calibrateCamera(world_points, image_pixels, (w, h), None, None)

print("\nRMS reprojection error:", error)
print("\ncamera matrix:\n", camera_matrix)
print("    fx =", camera_matrix[0, 0])
print("    fy =", camera_matrix[1, 1])
print("    cx =", camera_matrix[0, 2])
print("    cy =", camera_matrix[1, 2])
print("\ndistortion coefficients: ", dist_coefs.ravel())


## Build undistorted images

Now we can use the estimated distortion coefficients to undistort the image. In this exercise, the image distortion is very low, wo you will not see a drastic change.


In [ ]:
# JUST TUN THIS CELL

# undistort the image with the calibration, show before and after side by side
plt.figure(figsize=(13, 20))
for i, fn in enumerate(img_names):
    imgBGR = cv2.imread(fn)
    imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)

	# undistort the image using the computed camera parameters
    dst = cv2.undistort(imgRGB, camera_matrix, dist_coefs)

    if i < 4:
        plt.subplot(4, 2, i * 2 + 1)
        plt.imshow(imgRGB)
        plt.title("original image")
        plt.subplot(4, 2, i * 2 + 2)
        plt.imshow(dst)
        plt.title("undistorted image")

plt.show()

## Example for full projection from 3D to 2D of a cube


In [ ]:
# JUST TUN THIS CELL

# TODO: if you want, modify the 3D object size and corners!

n_squares_length = 3
objectPoints = (
    n_squares_length
    * checkerboard_square_size
    * np.array(
        [
            [0, 0, 0],
            [0, 1, 0],
            [1, 1, 0],
            [1, 0, 0],
            [0, 0, -1],
            [0, 1, -1],
            [1, 1, -1],
            [1, 0, -1],
        ]
    )
)


def draw(img, imgpts):
    imgpts = np.int32(imgpts).reshape(-1, 2)

    # draw ground floor in green
    img = cv2.drawContours(img, [imgpts[:4]], -1, (0, 255, 0), -1)

    # draw pillars in blue color
    for i, j in zip(range(4), range(4, 8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]), (0, 0, 255), 6)

    # draw top layer in red color
    img = cv2.drawContours(img, [imgpts[4:]], -1, (255, 0, 0), 6)

    return img


plt.figure(figsize=(20, 20))
for i, fn in enumerate(img_names):
    imgBGR = cv2.imread(fn)
    imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)

    imgpts = cv2.projectPoints(objectPoints, _rvecs[i], _tvecs[i], camera_matrix, dist_coefs)[0]
    drawn_image = draw(imgRGB, imgpts)

    if i < 12:
        plt.subplot(4, 3, i + 1)
        plt.imshow(drawn_image)

plt.show()
